In [1]:
import requests
import pandas as pd
# from pandas.io.json import json_normalize
from config import *
import sqlite3
from sqlite3 import Error

### Yelp

In [2]:
yelp_restaurant = requests.get('https://api.yelp.com/v3/businesses/search',
            params = {
                'location': 'Vancouver, BC',
                'term': 'restaurants'},
            headers = {'Authorization': f'Bearer {YELP_API}'})

In [3]:
yelp_restaurant.status_code

200

In [4]:
df_yelp = pd.json_normalize(yelp_restaurant.json(), 'businesses')
df_yelp.to_csv('data/yelp_restaurants.csv')

In [5]:
pop_yelp = df_yelp[['name', 'rating', 'review_count']]

### Top 10 Ratings

In [6]:
top10_rating_yelp = pop_yelp.head(10)
top10_rating_yelp.sort_values('rating', ascending=False)

,name,rating,review_count
0,Guu with Garlic,4.5,946
2,Saku Robson,4.5,214
4,Miku,4.5,1886
5,Fable Kitchen,4.5,693
6,Kingyo,4.5,889
7,Linh Cafe,4.5,17
1,The Flying Pig - Yaletown,4.0,1121
3,Saku Broadway,4.0,151
8,Phnom Penh,4.0,1355
9,Dinesty Dumpling House,4.0,805


### Bottom 10 Ratings

In [7]:
bot10_rating_yelp = pop_yelp.tail(10)
bot10_rating_yelp.sort_values('rating', ascending=False)

,name,rating,review_count
10,Fanny Bay Oyster Bar & Shellfish Market,4.5,603
11,Kosoo,4.5,28
12,Blue Water Cafe,4.5,939
19,So Hyang Korean Cuisine,4.5,283
13,Chambar,4.0,1397
14,Twisted Fork,4.0,1052
15,The Flying Pig - Gastown,4.0,943
16,Kokoro Tokyo Mazesoba,4.0,370
17,Medina Cafe,4.0,2365
18,Forage,4.0,645


### Summary

In [8]:
pop_yelp.describe()

,rating,review_count
count,20.000000,20.000000
mean,4.250000,835.100000
std,0.256495,607.156175
min,4.000000,17.000000
25%,4.000000,348.250000
50%,4.250000,847.000000
75%,4.500000,1069.250000
max,4.500000,2365.000000


### Top 10 Reviews

In [9]:
top10_review_yelp = pop_yelp.head(10)
top10_review_yelp.sort_values('review_count', ascending = False)

,name,rating,review_count
4,Miku,4.5,1886
8,Phnom Penh,4.0,1355
1,The Flying Pig - Yaletown,4.0,1121
0,Guu with Garlic,4.5,946
6,Kingyo,4.5,889
9,Dinesty Dumpling House,4.0,805
5,Fable Kitchen,4.5,693
2,Saku Robson,4.5,214
3,Saku Broadway,4.0,151
7,Linh Cafe,4.5,17


### Bottom 10 Reviews

In [10]:
bot10_review_yelp = pop_yelp.tail(10)
bot10_review_yelp.sort_values('review_count')

,name,rating,review_count
11,Kosoo,4.5,28
19,So Hyang Korean Cuisine,4.5,283
16,Kokoro Tokyo Mazesoba,4.0,370
10,Fanny Bay Oyster Bar & Shellfish Market,4.5,603
18,Forage,4.0,645
12,Blue Water Cafe,4.5,939
15,The Flying Pig - Gastown,4.0,943
14,Twisted Fork,4.0,1052
13,Chambar,4.0,1397
17,Medina Cafe,4.0,2365


### FourSquare

In [11]:
fsq_restaurant = requests.get('https://api.foursquare.com/v3/places/search',
                              params = {
                                  'near': 'Vancouver, BC',
                                  'categories': '13065',
                                  'limit': 50
                              },
                              headers = {
                                  'Accept': 'application/json',
                                  'Authorization': 'fsq3NBY1z05TqyRLxaFXhEmuBXlW4C+oCQ8jFzDNErgRGfs='
                              })

In [12]:
fsq_restaurant.status_code

200

In [13]:
df_fsq = pd.json_normalize(fsq_restaurant.json(), 'results')
df_fsq.to_csv('data/fsq_restaurant.csv')

## SQL

In [15]:
# create and connect to database
def create_connection(path):
    try:
        connection = sqlite3.connect(path)
        print('Connection to SQLite DB successful')
    except Error as e:
        print(f'The error "{e}" occurred')
    return connection

# execute customized query
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print('Query executed successfully')
    except Error as e:
        print(f'The error {e} occurred')

In [17]:
create_restaurant_table = '''
CREATE TABLE IF NOT EXISTS restaurant (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    review_count INTEGER,
    rating FLOAT,
    address_1 TEXT,
    address_2 TEXT,
    city TEXT,
    postal_code TEXT)
'''

In [16]:
# create connection
connection = create_connection("lhl_project_2.sqlite")

Connection to SQLite DB successful


In [18]:
execute_query(connection, create_restaurant_table)

Query executed successfully
